In [31]:
import cv2
import numpy as np
def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
        
    return index
###for source face###############
img = cv2.imread('./images/dwight.png')
cv2.imshow('dwight', img)
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
faces = detector(img_gray)
for face in faces:
    landmarks = predictor(img_gray, face)
    landmarks_points = []
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points.append((x, y))
#         cv2.circle(img, (x,y), 3, (0,255,0), -1)   
mask = np.zeros_like(img_gray)
# cv2.imshow('maskin',mask)
points = np.array(landmarks_points, np.int32)
convexhull = cv2.convexHull(points)
# cv2.polylines(img, [convexhull], True, (255, 0, 0), 3)
cv2.fillConvexPoly(mask, convexhull, 255)  
# cv2.imshow('mask',mask)
masked = cv2.bitwise_and(img,img,mask=mask)  
# cv2.imshow('image',img)
# cv2.imshow('maskedimg',masked)
rect = cv2.boundingRect(convexhull)
subdiv = cv2.Subdiv2D(rect)
subdiv.insert(landmarks_points)
triangles = subdiv.getTriangleList()
triangles = np.array(triangles, dtype=np.int32)
singletriangle = []
for t in triangles: 
    pt1 = (t[0], t[1])
    pt2 = (t[2], t[3])
    pt3 = (t[4], t[5])
    indexpt1 = np.where((points == pt1).all(axis=1))
    indexpt1 = extract_index_nparray(indexpt1)
    indexpt2 = np.where((points == pt2).all(axis=1))
    indexpt2 = extract_index_nparray(indexpt2)
    indexpt3 = np.where((points == pt3).all(axis=1))
    indexpt3 = extract_index_nparray(indexpt3)
    if indexpt1!=None and indexpt2!=None and indexpt2!=None:
        trianglecoord = [indexpt1,indexpt2,indexpt3]
        singletriangle.append(trianglecoord)
#     cv2.line(img, pt1, pt2, (0, 0, 255), 1)
#     cv2.line(img, pt2, pt3, (0, 0, 255), 1)
#     cv2.line(img, pt1, pt3, (0, 0, 255), 1)

#####for target face#########
targetface = cv2.imread('./images/targetimg.jpg')
cv2.imshow('jim',targetface)
# targetfacenew = targetface.copy()
(height,width,channels) = targetface.shape
newface = np.zeros((height,width,channels), np.uint8)
targetface_gray = cv2.cvtColor(targetface,cv2.COLOR_BGR2GRAY)
target = detector(targetface_gray)
for face in target:
    landmarks = predictor(targetface_gray, face)
    landmarks_points2 = []
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points2.append((x, y))
        points3 = np.array(landmarks_points2, np.int32)
        convexhull3 = cv2.convexHull(points3)
#         cv2.circle(targetface, (x,y), 3, (0,255,0), -1)

for temp in singletriangle: 
    ####face 1#######
    tr1pt1 = landmarks_points[temp[0]]
    tr1pt2 = landmarks_points[temp[1]]
    tr1pt3 = landmarks_points[temp[2]]   
    tr2pt1 = landmarks_points2[temp[0]]
    tr2pt2 = landmarks_points2[temp[1]]
    tr2pt3 = landmarks_points2[temp[2]]
#     cv2.line(img, tr1pt1, tr1pt2, (0, 0, 255), 1)
#     cv2.line(img, tr1pt2, tr1pt3, (0, 0, 255), 1)
#     cv2.line(img, tr1pt1, tr1pt3, (0, 0, 255), 1)
    tri1 = np.array([tr1pt1, tr1pt2, tr1pt3], np.int32)
    rect1 = cv2.boundingRect(tri1)
    (x,y,w,h) = rect1
    crpdtri1 = img[y:y+h,x:x+w]
    mask1 = np.zeros((h,w),np.uint8)
    temppts1 = np.array([[tr1pt1[0]-x, tr1pt1[1]-y],
                         [tr1pt2[0]-x, tr1pt2[1]-y],
                         [tr1pt3[0]-x, tr1pt3[1]-y]],np.int32)
    convexhull1 = cv2.convexHull(temppts1)
    cv2.fillConvexPoly(mask1, temppts1, 255)  
#     cv2.line(lines_space_mask, tr1pt1, tr1pt2, 255)
#     cv2.line(lines_space_mask, tr1pt2, tr1pt3, 255)
#     cv2.line(lines_space_mask, tr1pt1, tr1pt3, 255)
#     lines_space = cv2.bitwise_and(img, img, mask=lines_space_mask)
#     cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 1)
    ######face 2######

#     cv2.line(targetface, tr2pt1, tr2pt2, (0, 0, 255), 1)
#     cv2.line(targetface, tr2pt2, tr2pt3, (0, 0, 255), 1)
#     cv2.line(targetface, tr2pt1, tr2pt3, (0, 0, 255), 1)
    tri2 = np.array([tr2pt1, tr2pt2, tr2pt3], np.int32)
    rect2 = cv2.boundingRect(tri2)
    (x,y,w,h) = rect2
    crpdtri2 = targetface[y:y+h,x:x+w]
    
    mask2 = np.zeros((h,w),np.uint8)
    temppts2 = np.array([[tr2pt1[0]-x, tr2pt1[1]-y],
                        [tr2pt2[0]-x, tr2pt2[1]-y],
                        [tr2pt3[0]-x, tr2pt3[1]-y]],np.int32)
    convexhull2 = cv2.convexHull(temppts2)
    cv2.fillConvexPoly(mask2, temppts2, 255)  
    crpdtri2 = cv2.bitwise_and(crpdtri2,crpdtri2,mask=mask2)
    temppts1 = np.float32(temppts1)
    temppts2 = np.float32(temppts2)
    M = cv2.getAffineTransform(temppts1, temppts2)
    warped_triangle = cv2.warpAffine(crpdtri1, M, (w,h))
#     cv2.imshow('war',warped_triangle)
    warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=mask2)
    #for no overlapping of image next two linw is useful \
#     cv2.imshow('warpednew',warped_triangle)
#     cv2.imshow('mask',mask2)
#     kernel = np.ones((3,3), np.uint8)
#     warped_triangle = cv2.dilate(warped_triangle,kernel,iterations = 1)
    triarea = newface[y:y+h,x:x+w]
    triarea_gray = cv2.cvtColor(triarea, cv2.COLOR_BGR2GRAY)
    _,maskedtri = cv2.threshold(triarea_gray, 1, 255, cv2.THRESH_BINARY_INV)
    warped_triangle = cv2.bitwise_and(warped_triangle,warped_triangle, mask = maskedtri)
    triarea = cv2.add(triarea, warped_triangle)
    newface[y:y+h,x:x+w] = triarea


#####Applying new face######

facemask = np.zeros_like(newface)
gray_face = cv2.cvtColor(newface,cv2.COLOR_BGR2GRAY)
_,facemask0 = cv2.threshold(gray_face,1,255,cv2.THRESH_BINARY)
_,facemask = cv2.threshold(gray_face,1,255,cv2.THRESH_BINARY_INV)
# cv2.imshow('facemas',facemask)
result = cv2.bitwise_and(targetface, targetface, mask=facemask)
# cv2.imshow('res',result)
result = cv2.add(result,newface)
# cv2.imshow('newres',result)

(x, y, w, h) = cv2.boundingRect(convexhull3)
center_face2 = (int((x + x + w) / 2), int((y + y + h) / 2))
seamlessclone = cv2.seamlessClone(result, targetface, facemask0, center_face2, cv2.NORMAL_CLONE)

# cv2.imshow('newface',newface)  

cv2.imshow('IDENTITY CRISIS',seamlessclone)
cv2.waitKey(0)
cv2.destroyAllWindows()